In [1]:
import os
import whisper
import openai
import requests_html
import dataset
import pprint

from dotenv import load_dotenv
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

In [3]:
navegador = Chrome(service=Service(ChromeDriverManager().install()))

In [4]:
navegador.maximize_window()

In [5]:
navegador.get("https://pyvideo.org/index.html")

In [6]:
links = navegador.find_elements(By.XPATH, '//article/a')

In [8]:
for link in links:
    print(link.get_attribute('href'))

https://pyvideo.org/chipy/Exploring_the_Python_Run_Time_Environment.html
https://pyvideo.org/chipy/JSON_Web_Tokens_for_Fun_and_Profit.html
https://pyvideo.org/chipy/Ellipses_and_Arcane_Syntax.html
https://pyvideo.org/chipy/Micropython_gpio.html
https://pyvideo.org/chipy/Learning_Sprint_An_Experiment.html
https://pyvideo.org/pytexas-2023/day-2-keynote-full-stack-python.html


In [9]:
links[0].click()

In [10]:
links_video = navegador.find_elements(By.LINK_TEXT, 'ARCHIVE')

In [11]:
for link_video in links_video:
    print(link_video.get_attribute('href'))

https://pyvideo.org/chipy/Exploring_the_Python_Run_Time_Environment.html#archive
https://archive.org/details/June_23-Exploring_the_Python_Run_Time_Environment/Exploring_the_Python_Run_Time_Environment.mp4


In [12]:
video_url = links_video[1].get_attribute('href')
video_url

'https://archive.org/details/June_23-Exploring_the_Python_Run_Time_Environment/Exploring_the_Python_Run_Time_Environment.mp4'

In [15]:
navegador.save_screenshot('pagina.png')

True

In [13]:
sessao = requests_html.HTMLSession()

In [14]:
resposta = sessao.get(video_url)

In [16]:
video_download = resposta.html.xpath('//a[contains(text(), "MPEG4")]/@href', first=True)
video_download

'/download/June_23-Exploring_the_Python_Run_Time_Environment/Exploring_the_Python_Run_Time_Environment.mp4'

In [ ]:
resposta = sessao.get(f'https://archive.org{video_download}')
with open('video.mp4', 'wb') as arquivo:
    arquivo.write(resposta.content)

In [17]:
modelo = whisper.load_model("small")

In [ ]:
ffmpeg_extract_subclip("video.mp4", 0, 120, targetname="video_cortado.mp4")

In [19]:
transcricao = modelo.transcribe('video_cortado.mp4')

/home/gileno/workspace/nxtweb/venv/lib/python3.10/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [20]:
pprint.pprint(transcricao)

{'language': 'en',
 'segments': [{'avg_logprob': -0.18204337559389264,
               'compression_ratio': 1.5848214285714286,
               'end': 5.5600000000000005,
               'id': 0,
               'no_speech_prob': 0.05237644910812378,
               'seek': 0,
               'start': 0.0,
               'temperature': 0.0,
               'text': " All right, so I guess I'm going to start with my "
                       'motivation for the talk.',
               'tokens': [50364,
                          1057,
                          558,
                          11,
                          370,
                          286,
                          2041,
                          286,
                          478,
                          516,
                          281,
                          722,
                          365,
                          452,
                          12335,
                          337,
                          264,
    

In [21]:
pprint.pprint(transcricao['text'])

(" All right, so I guess I'm going to start with my motivation for the talk. "
 'So one of the things that always kind of confused me about Python is you '
 'write something in Python and then you click run and then the computer does '
 'some magic and it pops out an answer for you. And when I was looking through '
 'the C-Python interpreter, the core system that actually runs Python, the '
 'thing that conceptually gave me the hardest challenge of understanding was '
 "the runtime environment. So I'm going to start with a brief overview of what "
 "the C-Python interpreter is and what it does. So let's say I'm a clever, "
 'lazy programmer who was given the assignment that they need a program that '
 'is completely secure and bug-free. So I was like, if I just delete '
 "everything, then it's perfectly bug-free and secure. So how C-Python works "
 "is that there's basically two parts. There's the compiler that converts any "
 "Python code into a byte code and then there's the runtime,

In [22]:
load_dotenv()

True

In [23]:
openai.api_key = os.environ['OPENAI_KEY']

In [24]:
mensagens = []
mensagens.append(
    {"role": "user", "content": f"traduza o seguinte de inglês para português: {transcricao['text']}"},
)
chat = openai.ChatCompletion.create(
    model="gpt-3.5-turbo", messages=mensagens
)
chat['choices'][0]['message']['content']

'Certo, então acho que vou começar com a minha motivação para a palestra. Uma das coisas que sempre me confundiu um pouco sobre o Python é que você escreve algo em Python, clica em "executar" e o computador faz algumas mágicas e te dá uma resposta. E quando eu estava olhando o interpretador C-Python, o sistema central que realmente executa o Python, o que me deu mais dificuldade de entender do ponto de vista conceitual foi o ambiente de execução. Então vou começar com uma visão geral breve de o que é o interpretador C-Python e o que ele faz. Digamos que eu sou um programador esperto e preguiçoso a quem foi dado o trabalho de criar um programa completamente seguro e sem bugs. Então pensei, se eu simplesmente deletar tudo, ele ficará perfeitamente livre de bugs e seguro. A forma como o C-Python funciona é basicamente em duas partes. Há o compilador que converte qualquer código Python em um byte code, e depois há o tempo de execução, que é o que realmente executa o byte code e o código Py

In [ ]:
opcoes = Options()
opcoes.add_argument('--headless')
navegador = Chrome(options=opcoes)